In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import api_key

In [ ]:
#TK Code

In [ ]:
print ("I made changes to the file")
print("Added new print stmt")
print("last stmt")

In [ ]:
#TK code ends

In [ ]:
#Adam code

In [ ]:
#Presidential file to dataframe called df
file = "Resources/1976-2016-president.tab"
df = pd.read_csv(file,sep='\t',header=(0))
df.tail(40)

In [ ]:
#just curious about write in candidates
just_write_in = df.loc[df["writein"] == True]
just_write_in

In [ ]:
write_in_by_state = just_write_in.groupby("state")
write_in_by_state = pd.DataFrame(write_in_by_state["candidatevotes"].sum())
sorted_write_in_by_state = write_in_by_state["candidatevotes"].sort_values(ascending=False)
sorted_write_in_by_state

In [ ]:
candidate = df.groupby("candidate")
temp = candidate.sum()
temp["% of totalvotes"] = temp["candidatevotes"]/temp["totalvotes"]
temp = pd.DataFrame(temp["% of totalvotes"].sort_values(ascending=False))
temp.head(25)

In [ ]:
year_2016 = df.loc[df["year"]==2016]

by_candidate_2016 = year_2016.groupby("candidate")
total_votes = by_candidate_2016["candidatevotes"].sum().sort_values(ascending=False)
total_votes

In [ ]:
year_2012 = df.loc[df["year"]==2012]

by_candidate_2012 = year_2012.groupby("candidate")
total_votes_2012 = by_candidate_2012["candidatevotes"].sum().sort_values(ascending=False)
total_votes_2012

In [ ]:
#Adam code ends

In [ ]:
#Ryan code

In [ ]:
from census import Census
from us import states

c = Census(api_key, year=2018)

In [ ]:
* As only civilians are legel to vote, it would make sense to only pull the civilian employment data.
* The labor force is the sum of employed and unemployed persons
* Columns data to pull: 
* Income variables: 
    "income": B19013_001E, "income_per_capita": B19301_001E
* Employment variables:
    "employable civilians": B23025_003E, "employed civilians": B23025_004E, "unemployed civilians": B23025_005E
* Population variables: 
    "Total population": B01003_001E
* Poverty variables:
    "poverty people": "B17001_002E"
* Age variables:
    "median age": "B01002_001E" 

In [ ]:
raw_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B23025_003E',
                       'B23025_004E', 'B23025_005E', 'B01003_001E', "B01002_001E"), {'for':'state:*'})
df = pd.DataFrame(raw_data)
df = df.rename(columns = {'state': 'State#',
                          'NAME': 'State', 
                          'B01003_001E':'Total_population',
                          'B23025_003E':'Employable_civilians',
                          'B23025_004E':'Employed_civilians', 
                          'B23025_005E':'Unemployed_civilians', 
                          'B19013_001E':'Income',
                          'B19301_001E':'Income_per_capita',
                          "B01002_001E":'median age'
                          
    
})
df = df.sort_values('State#').reset_index(drop=True)
df

In [ ]:
#Ryan code ends

In [ ]:
#Connor code

In [ ]:
# Read in CSV Files
census_2012_df = pd.read_csv('Resources/2012_population_data.csv')
census_2016_df = pd.read_csv('Resources/2016_population_data.csv')

In [ ]:
census_2012_df.head()

In [ ]:
total_young_males = census_2012_df["Males 18 and 19 years"] + census_2012_df["Males 20 to 24"] + census_2012_df["Males 25 to 29 years"]
total_young_females = census_2012_df["Females 18 and 19 years"] + census_2012_df["Females 20 to 24"] + census_2012_df["Females 25 to 29 years"]
percent_males = total_young_males/census_2012_df["Total State Population"]
percent_females = total_young_females/census_2012_df["Total State Population"]
total_votes = census_2012_df["Democrat votes 2012"] + census_2012_df["Republican votes 2012"]
new_2012_df = pd.DataFrame({"State": census_2012_df["State"],
                            "Total Males 18 to 29":total_young_males,
                            "Total Females 18 to 29":total_young_females,
                            "Total People 18 to 29":total_young_males+total_young_females,
                            "Percent Males": percent_males*100,
                            "Percent Females": percent_females*100,
                            "Percent 18 to 29": (percent_males + percent_females)*100,
                            "Total Votes": total_votes,
                            "Percent Democrat": (census_2012_df["Democrat votes 2012"]/total_votes)*100,
                            "Percent Republican": (census_2012_df["Republican votes 2012"]/total_votes)*100
})



new_2012_df.head()

In [ ]:
plt.bar(new_2012_df["State"],new_2012_df["Percent 18 to 29"])
plt.title("2012 18 to 29 y/o by State")
plt.xlabel("State")
plt.ylabel("Percent of 18 to 29 y/o")
plt.ylim(new_2012_df["Percent 18 to 29"].min()-1,new_2012_df["Percent 18 to 29"].max()+1)
plt.xticks(rotation=90)
plt.tight_layout

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = linregress(new_2012_df["Percent 18 to 29"], new_2012_df["Percent Democrat"])
regress_values_d_2012 = new_2012_df["Percent 18 to 29"] * slope + intercept
line_eq_d_2012 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

(slope, intercept, rvalue, pvalue, stderr) = linregress(new_2012_df["Percent 18 to 29"], new_2012_df["Percent Republican"])
regress_values_r_2012 = new_2012_df["Percent 18 to 29"] * slope + intercept
line_eq_r_2012 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


In [ ]:
plt.scatter(new_2012_df["Percent 18 to 29"],new_2012_df["Percent Democrat"],marker='o', c='blue')
plt.scatter(new_2012_df["Percent 18 to 29"],new_2012_df["Percent Republican"],marker='o', c='red')
plt.xlabel("Percentage of 18 to 29")
plt.ylabel("Percentage of Votes")
plt.title("Voter type vs 18 to 29 y/o")
correlation_2012 = st.pearsonr(new_2012_df["Percent 18 to 29"],new_2012_df["Percent Democrat"])
plt.plot(new_2012_df["Percent 18 to 29"],regress_values_d_2012,"r-")
plt.plot(new_2012_df["Percent 18 to 29"],regress_values_r_2012,"r-",c="blue")
#plt.annotate(line_eq_d_2012,(14,24),fontsize=15,color="red")
ttest_2012 = st.ttest_ind(new_2012_df["Percent 18 to 29"],new_2012_df["Percent Democrat"],equal_var=False)
print(f"The correlation between both factors is {round(correlation_2012[0],2)}")
print(f'The regression line for Democrats is {line_eq_d_2012}.')
print(f'The regression line for Republicans is {line_eq_r_2012}.')
ttest_2012

In [ ]:
census_2016_df.head()

In [ ]:
total_young_males_2016 = census_2016_df["Males 18 and 19 years"] + census_2016_df["Males 20 to 24"] + census_2016_df["Males 25 to 29 years"]
total_young_females_2016 = census_2016_df["Females 18 and 19 years"] + census_2016_df["Females 20 to 24"] + census_2016_df["Females 25 to 29 years"]
percent_males_2016 = total_young_males_2016/census_2016_df["Total State Population"].astype('float')
percent_females_2016 = total_young_females_2016/census_2016_df["Total State Population"].astype('float')
total_votes = census_2012_df["Democrat votes 2012"] + census_2012_df["Republican votes 2012"]
new_2016_df = pd.DataFrame({"State": census_2016_df["State"],
                            "Total Males 18 to 29": total_young_males,
                            "Total Females 18 to 29": total_young_females,
                            "Total People 18 to 29": total_young_males_2016 + total_young_females_2016,
                            "Percent Males": percent_males_2016*100,
                            "Percent Females": percent_females_2016*100,
                            "Percent 18 to 29": (percent_males_2016 + percent_females_2016)*100,
                            "Total Votes": total_votes,
                            "Percent Democrat": (census_2012_df["Democrat votes 2012"]/total_votes)*100,
                            "Percent Republican": (census_2012_df["Republican votes 2012"]/total_votes)*100
})




new_2016_df.head()

In [ ]:
plt.bar(new_2016_df["State"],new_2016_df["Percent 18 to 29"])
plt.title("2016 18 to 29 y/o by State")
plt.xlabel("State")
plt.ylabel("Percent of 18 to 29 y/o")
plt.ylim(new_2016_df["Percent 18 to 29"].min()-1,new_2016_df["Percent 18 to 29"].max()+1)
plt.xticks(rotation=90)
plt.tight_layout

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = linregress(new_2016_df["Percent 18 to 29"], new_2016_df["Percent Democrat"])
regress_values_d_2016 = new_2016_df["Percent 18 to 29"] * slope + intercept
line_eq_d_2016 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

(slope, intercept, rvalue, pvalue, stderr) = linregress(new_2016_df["Percent 18 to 29"], new_2016_df["Percent Republican"])
regress_values_r_2016 = new_2016_df["Percent 18 to 29"] * slope + intercept
line_eq_r_2016 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

In [ ]:
plt.scatter(new_2016_df["Percent 18 to 29"],new_2016_df["Percent Democrat"],marker='o', c='blue')
plt.scatter(new_2016_df["Percent 18 to 29"],new_2016_df["Percent Republican"],marker='o', c='red')
plt.xlabel("Percentage of 18 to 29")
plt.ylabel("Percentage of Votes")
plt.title("Voter type vs 18 to 29 y/o")
correlation_2016 = st.pearsonr(new_2016_df["Percent 18 to 29"],new_2016_df["Percent Democrat"])
plt.plot(new_2016_df["Percent 18 to 29"],regress_values_d_2016,"r-")
plt.plot(new_2016_df["Percent 18 to 29"],regress_values_r_2016,"r-",c="blue")
#plt.annotate(line_eq_d_2012,(14,24),fontsize=15,color="red")
ttest_2016 = st.ttest_ind(new_2016_df["Percent 18 to 29"],new_2016_df["Percent Democrat"],equal_var=False)
print(f"The correlation between both factors is {round(correlation_2016[0],2)}")
print(f'The regression line for Democrats is {line_eq_d_2016}.')
print(f'The regression line for Republicans is {line_eq_r_2016}.')
ttest_2016

In [ ]:
st.ttest_ind(new_2016_df["Percent 18 to 29"],new_2016_df["Percent Democrat"],equal_var=False)

In [ ]:
#Connor code ends